In [ ]:
# default_exp matching

# Matching

> Functions related to matching

This notebook contains all functions related to matching.

Current ToDo here:

- Move the current Code to here

## Matching MS1 to MS2

Matching MS1 features to MS2 spectra

In [6]:
#export

from sklearn.neighbors import KDTree
import pandas as pd
import numpy as np


def match_ms2(feature_table, query_data, ppm_range = 20, rt_range = 0.5, n_neighbors=3):
    """
    Match MS1 features to MS2 based on rt and mz
    """
    tree_points = feature_table[['mz','rt_apex']].values

    tree_points[:,0] = np.log(tree_points[:,0])*1e6/ppm_range #m/z -> log transform, this is in ppm then
    tree_points[:,1] = tree_points[:,1]/rt_range # -> this is in minutes

    matching_tree = KDTree(tree_points, metric="minkowski")

    query_mz = np.log(query_data['mono_mzs2'])*1e6/ppm_range  
    query_rt = query_data['rt_list_ms2'] / rt_range

    ref_points = np.array([query_mz, query_rt]).T

    ref_points[ref_points == -np.inf] = 0
    ref_points[ref_points == np.inf] = 0

    dist, idx = matching_tree.query(ref_points, k=n_neighbors)    

    all_df = []
    for neighbor in range(n_neighbors):
        ref_df = pd.DataFrame(np.array([query_data['rt_list_ms2'], query_data['prec_mass_list2'], query_data['mono_mzs2'], query_data['charge2']]).T, columns=['rt', 'mass', 'mz', 'charge'])

        ref_df['mass_matched'] = feature_table.iloc[idx[:,neighbor]]['mass'].values
        ref_df['mass_offset'] = ref_df['mass_matched'] - ref_df['mass']

        ref_df['rt_matched'] = feature_table.iloc[idx[:,neighbor]]['rt_apex'].values
        ref_df['rt_offset'] = ref_df['rt_matched'] - ref_df['rt']
        
        ref_df['mz_matched'] = feature_table.iloc[idx[:,neighbor]]['mz'].values
        ref_df['mz_offset'] = ref_df['mz_matched'] - ref_df['mz']

        ref_df['charge_matched'] = feature_table.iloc[idx[:,neighbor]]['charge'].values

        ref_df['query_idx'] = ref_df.index
        ref_df['feature_idx'] = idx[:,neighbor]

        ref_df['int_apex'] = feature_table.iloc[idx[:,neighbor]]['int_apex'].values
        ref_df['int_sum'] = feature_table.iloc[idx[:,neighbor]]['int_sum'].values

        ref_df['dist'] = dist[:,neighbor]

        ref_df = ref_df[ref_df['dist']<1]

        all_df.append(ref_df)

    features = pd.concat(all_df)
    
    features = features.sort_values('mass_matched', ascending=True)
    features = features.reset_index(drop=True)
    
    return features

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_settings.ipynb.
Converted 12_settings_template.ipynb.
Converted index.ipynb.
Converted MaxQuant Comparison.ipynb.
Converted MaxQuant Comparison_v2.ipynb.
Converted Runner.ipynb.
